In [89]:
import pandas as pd
import numpy as np
from ast import literal_eval

df = pd.read_csv('export_dataframe2.csv',sep=',',converters={'hashtag':literal_eval})

In [90]:
df = df[['user_id','hashtag']]
df[:5]

,user_id,hashtag
0,10836,"[techsailor, officewarming]"
1,10836,"[techsailor, officewarming]"
2,10836,"[techsailor, officewarming]"
3,10836,"[techsailor, officewarming]"
4,10836,"[techsailor, officewarming]"


# Average hashtags per tweet

In [81]:
df_avg_hashtag = df.copy()
df_avg_hashtag['length_of_hashtag']=df_avg_hashtag['hashtag'].apply(lambda x:len(x) if x!=len(x) else 0)
df_avg_hashtag.head()

,user_id,hashtag,length_of_hashtag
0,10836,"[techsailor, officewarming]",2
1,10836,"[techsailor, officewarming]",2
2,10836,"[techsailor, officewarming]",2
3,10836,"[techsailor, officewarming]",2
4,10836,"[techsailor, officewarming]",2


In [82]:
df_avg_hashtag=df_avg_hashtag.drop('hashtag',axis=1)
df_avg_hashtag=df_avg_hashtag.rename(columns={'length_of_hashtag':'avg_hashtags'})
df_avg_hashtag_avg = df_avg_hashtag.groupby('user_id').mean()
df_avg_hashtag_avg.head()

,avg_hashtags
user_id,
1038,1.500000
1437,2.250000
3148,1.000000
9375,7.000000
10336,1.333333


# Duplicate Hashtags

In [107]:
df_duplicate = df.copy()
df_duplicate['n_tweet'] = '1'
df_duplicate = df_duplicate.groupby('user_id').sum()
df_duplicate['n_tweet'] = df_duplicate['n_tweet'].apply(lambda x:len(x))
df_duplicate['n_hashtag'] = df_duplicate['hashtag'].apply(lambda x:len(x))
df_duplicate.head()

,hashtag,n_tweet,n_hashtag
user_id,,,
1038,"[2012, paniekpeeuw, fail]",2,3
1437,"[wecoolandallbut, losemynumber, netenberg, fan...",4,9
3148,[xfactor],1,1
9375,"[Rights, of, man, Martin, Wynne, celtic, music...",2,14
10336,"[ff, hcr, hcr, vegan, blogher, hcr, vegan, hcr]",6,8


In [108]:
def calculate_duplicate(hashtags):
    n_duplicate = len(hashtags) - len(set(hashtags))
    return n_duplicate

In [110]:
df_duplicate['n_duplicate'] = df_duplicate['hashtag'].apply(lambda x:calculate_duplicate(x))
df_duplicate['ratio_duplicate'] = df_duplicate['n_duplicate']/(df_duplicate['n_tweet']*df_duplicate['n_hashtag'])
df_duplicate.head()

,hashtag,n_tweet,n_hashtag,n_duplicate,ratio_duplicate
user_id,,,,,
1038,"[2012, paniekpeeuw, fail]",2,3,0,0.000000
1437,"[wecoolandallbut, losemynumber, netenberg, fan...",4,9,1,0.027778
3148,[xfactor],1,1,0,0.000000
9375,"[Rights, of, man, Martin, Wynne, celtic, music...",2,14,7,0.250000
10336,"[ff, hcr, hcr, vegan, blogher, hcr, vegan, hcr]",6,8,4,0.083333


# Common topic

In [10]:
from functools import reduce

def Union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 

def common_topic(series):
    return reduce(lambda x, y: Union(x,y), series)

In [25]:
df = df.groupby('user_id').agg({'hashtag': common_topic})
df.head()

,hashtag
user_id,
1038,"[paniekpeeuw, 2012, fail]"
1437,"[ff, Reds, netenberg, losemynumber, fantastico..."
3148,[xfactor]
9375,"[Wynne, celtic, of, man, Martin, music, Rights]"
10336,"[ff, vegan, blogher, hcr]"


### No of common topics between u_i,u_j   /  Total No of topics of u_(i )+Total No of topics of u_j

In [30]:
def radio_common_topic(df,user1,user2):
    x1 = df.loc[user1]['hashtag']
    x2 = df.loc[user2]['hashtag']
    intersection = len(set(x1)&set(x2))
    union = len(x1)+len(x2)
    print("Score: " + str(intersection/union))
    return intersection/union

In [31]:
radio_common_topic(df,1437,304713)

Score: 0.0625


0.0625